In [1]:
import tushare as ts
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import pandas as pd
import numpy as np
import baostock as bao
from baostock.data.resultset import ResultData
import baostock.common.contants as bao_consts
from pandas import Timestamp
from sz.stock_data.toolbox.data_provider import ts_code
from datetime import date, datetime, timedelta

ts.set_token('f96b1eeee9c8fddd357f2299cdedc1c88b2bb2a30ae1f772cf810dea')
ts_api = ts.pro_api()

blg = bao.login()
print('baostock login => error_code: %s error_msg: %s\n' %
      (blg.error_code, blg.error_msg))


from sz.stock_data.stock_data import StockData
StockData().setup(data_dir = '/Volumes/USBDATA/stock_data')

from sz.stock_data.stocks.stock_daily import StockDaily
from sz.stock_data.toolbox.helper import need_update_by_trade_date

 

/Users/kk/miniconda3/envs/quant/lib/python3.7/site-packages/tushare/util/upass.py:28: FutureWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated



login success!
baostock login => error_code: 0 error_msg: success



In [2]:
stock = StockDaily(StockData().data_dir, stock_code = '300059.SZ').prepare()

print("need update: %s" % need_update_by_trade_date(stock.dataframe, 'date'))

need update: True


In [2]:
for code in StockData().hs300.stock_codes():
    print(code)


000001.SZ
000002.SZ
000063.SZ
000069.SZ
000100.SZ
000157.SZ
000166.SZ
000333.SZ
000338.SZ
000402.SZ
000408.SZ
000413.SZ
000415.SZ
000423.SZ
000425.SZ
000538.SZ
000553.SZ
000568.SZ
000596.SZ
000625.SZ
000627.SZ
000629.SZ
000630.SZ
000651.SZ
000656.SZ
000661.SZ
000671.SZ
000703.SZ
000709.SZ
000725.SZ
000728.SZ
000768.SZ
000776.SZ
000783.SZ
000786.SZ
000858.SZ
000876.SZ
000895.SZ
000898.SZ
000938.SZ
000961.SZ
000963.SZ
001979.SZ
002001.SZ
002007.SZ
002008.SZ
002010.SZ
002024.SZ
002027.SZ
002032.SZ
002044.SZ
002050.SZ
002065.SZ
002081.SZ
002120.SZ
002142.SZ
002146.SZ
002153.SZ
002179.SZ
002202.SZ
002230.SZ
002236.SZ
002241.SZ
002252.SZ
002271.SZ
002294.SZ
002304.SZ
002310.SZ
002311.SZ
002352.SZ
002410.SZ
002411.SZ
002415.SZ
002422.SZ
002456.SZ
002460.SZ
002466.SZ
002468.SZ
002475.SZ
002493.SZ
002508.SZ
002555.SZ
002558.SZ
002594.SZ
002601.SZ
002602.SZ
002624.SZ
002625.SZ
002673.SZ
002714.SZ
002736.SZ
002739.SZ
002773.SZ
002925.SZ
002938.SZ
002939.SZ
002945.SZ
300003.SZ
300015.SZ
300017.SZ


In [3]:
ts_api.margin_detail(
    start_date = '20190925',
    end_date = '20190925'
)

,trade_date,ts_code,rzye,rqye,rzmre,rqyl,rzche,rqchl,rqmcl,rzrqye


In [10]:
bao.query_history_k_data_plus(
    code = '603259.SH',
    fields = 'date,time,code,open,high,low,close,volume,amount,adjustflag',
    start_date = '2019-09-23',
    end_date = '2019-09-23',
    frequency = '5'
).get_data()

,date,time,code,open,high,low,close,volume,amount,adjustflag
0,2019-09-23,20190923093500000,sh.603259,84.2000,84.3200,83.1100,83.2000,347600,29048400.0000,3
1,2019-09-23,20190923094000000,sh.603259,83.1900,83.7000,82.7800,83.7000,447799,37224558.0000,3
2,2019-09-23,20190923094500000,sh.603259,83.6900,83.7000,82.9900,83.2700,204880,17077204.0000,3
3,2019-09-23,20190923095000000,sh.603259,83.2900,83.5200,82.9000,83.4600,249745,20759794.0000,3
4,2019-09-23,20190923095500000,sh.603259,83.4600,83.4600,83.1300,83.1900,228270,19008604.0000,3
5,2019-09-23,20190923100000000,sh.603259,83.1900,83.2000,82.7600,82.7600,332124,27566168.0000,3
6,2019-09-23,20190923100500000,sh.603259,82.7600,83.1800,82.5900,83.1500,231369,19179096.0000,3
7,2019-09-23,20190923101000000,sh.603259,83.1500,83.1500,82.9000,83.0200,191105,15866150.0000,3
8,2019-09-23,20190923101500000,sh.603259,83.0000,83.1000,82.9200,83.0300,181362,15054528.0000,3
9,2019-09-23,20190923102000000,sh.603259,83.0000,83.1100,83.0000,83.1000,99100,8229654.0000,3


In [11]:
a = '200509'
pd.to_datetime(a, format = '%Y%m')

Timestamp('2005-09-01 00:00:00')